In [1]:
import jieba
import json
import re
import os
import re
import chardet

In [31]:
# 读取数据集，预处理。
# 分句子
# 句子分词

# 文件批处理
def FileReader(tag):
    # print(os.getcwd())
    path = r'../ex_5/data/htl_del_4000/' + tag 
    os.listdir(path)
    FileList = []
    for iter in os.listdir(path):
        if os.path.splitext(iter)[1] == '.txt':
            FileList.append(iter)
    return FileList


def _PreProcess(article):
    article = re.sub('\\s','',article)
    article = re.sub('，','',article)
    article = re.sub('；','',article)
    article = re.sub('[a-z,A-Z]*','',article)
    # print(article)
    sentence = re.split('。|？|！',article)
    # if len(sentence) > 1:
    # print(sentence)
    row_list =[]
    for iter in sentence:
        row_list.append(jieba.lcut(iter,cut_all = False)) 
    
    return row_list

def PreProcess(path,id_f):
    text = ''
    # print(path)
    with open(path,'r',encoding = 'gbk',errors = 'ignore') as txt:
        text = txt.readline()
    # print(text[:20])
    sentences = _PreProcess(text)
    # print(type(sentences))
    dic = {}
    id_c = 0
    for iter in sentences:
        dic[id_c] = iter
        id_c += 1
    return dic



def File_Dic(FileList,tag):
    path = r'../ex_5/data/htl_del_4000/' + tag + r'/'
    dic = {}

    # for iter in range(0,3):
    for iter in range(0,len(FileList)):
        cur_path = path + FileList[iter]
        dic[iter] = PreProcess(cur_path,iter)
        # print(dic[iter])
    return dic


def row_split(tag):
    FileList = FileReader(tag)
    volc = File_Dic(FileList,tag)
    j_data = json.dumps(volc,ensure_ascii=False)
    with open(r'./row_语料分词_' + tag + '.json' ,'w',encoding='utf-8') as f:
        f.write(j_data)

row_split('pos')
row_split('neg')

In [27]:
# import re
# s = '早餐太差无论去多少人那边也不加食品的。酒店应该重视一下这个问题了。'
# s = re.split('。|，',s)
# print(s)

['早餐太差无论去多少人那边也不加食品的', '酒店应该重视一下这个问题了', '']


In [32]:
# 把所有的句子load 进来
def loadSentence():
    posLen = 0
    negLen = 0
    sentence = []
    dic = {}
    path = '.\\row_语料分词_'
    with open(path + 'pos.json','r',encoding = 'utf-8') as f:
        dic = json.load(f)
        for iter in dic.values():
            for val in iter.values():
                sentence.append(val)
                posLen += 1
    with open(path + 'neg.json','r',encoding = 'utf-8') as f:
        dic = json.load(f)
        for iter in dic.values():
            for val in iter.values():
                sentence.append(val)
                negLen += 1
    return sentence,posLen,negLen
sentences,posLen,negLen = loadSentence()


In [33]:
# print(len(sentences))

12361


In [34]:
# print(sentences[:5])

[['早餐', '太', '差', '无论', '去', '多少', '人', '那边', '也', '不加', '食品', '的'], ['酒店', '应该', '重视', '一下', '这个', '问题', '了'], [], ['宾馆', '在', '小', '街道', '上', '不大好', '找', '但', '还好', '北京', '热心', '同胞', '很多', '~'], ['中心', '周围', '没什么', '店铺', '说', '5', '星', '有点', '勉强', '.', '不', '知道', '为什么', '卫生间', '没有', '电吹风']]


In [35]:
from gensim.models import word2vec

CBOW_Model = word2vec.Word2Vec(sentences, vector_size=64,sg = 0)
CBOW_Model.save("word2vec_CBOW.model")


skip_garm_Model = word2vec.Word2Vec(sentences, vector_size=64,sg = 0)
skip_garm_Model.save("word2vec_skip_garm.model")


In [36]:
# 训练好的 词向量
# https://radimrehurek.com/gensim/models/keyedvectors.html#
from gensim.models import KeyedVectors

# 二进制形式
# CBOW_vectors = model.wv
# CBOW_vectors.save('CBOWvectors.kv')
# CBOW_word_vectors = KeyedVectors.load('vectors.kv')

CBOW_Model.wv.save_word2vec_format('CBOW.txt',binary = False)
skip_garm_Model.wv.save_word2vec_format('skip_garm.txt',binary = False)


In [37]:
#读取模型
# model = word2vec.Word2Vec.load("your_model_name")
CBOW_Vector = CBOW_Model.wv
skip_garm_Vector = skip_garm_Model.wv
print(CBOW_Vector['干净'])

print(CBOW_Vector.most_similar('干净'))
print()
print(skip_garm_Vector.most_similar('差'))

[ 0.7268715   0.35475895  0.18004695 -0.06718917 -0.09091606 -0.02024413
  0.87938905  0.9853866  -0.4175484   0.4860955   0.51452655 -0.98285997
 -0.5241011  -0.6611428   0.47952646  0.5127956   0.43668288 -0.4351187
 -0.54587764  1.6200808   0.70000196  1.0937848   0.24367997 -1.1306806
  0.18492514  0.6865539  -0.07901439 -0.1192142   0.02157725  0.0179108
 -0.49417496  0.43541566 -0.79560816 -0.8349246  -0.8719583   0.7420447
  0.9168249   0.6524112   0.6179147  -0.5001665  -0.52447957  1.1290827
 -0.05631541 -1.0157641   0.30878562 -0.47939503  0.13565885  0.06868114
 -0.04340766 -0.11992697  0.32738692  0.5200104   0.29660487  1.214728
  1.4410632  -0.25559792 -0.09852278  0.24846295 -0.51085925 -0.91901654
 -1.0375206  -0.8896146  -0.48458543  0.43686476]
[('旧', 0.9942488074302673), ('一般', 0.993561327457428), ('陈旧', 0.9916391372680664), ('设施', 0.987526535987854), ('环境', 0.9807381629943848), ('比较', 0.9787128567695618), ('隔音', 0.9759792685508728), ('较', 0.9726209044456482), ('但', 

In [38]:
# 利用已得到的词的向量表示做句子的向量表示
print(len(CBOW_Vector))
print(len(skip_garm_Vector))

3113
3113


In [39]:
print(sentences[0])
print('早餐' in CBOW_Vector)
print(CBOW_Vector['早餐'] * 2)

['早餐', '太', '差', '无论', '去', '多少', '人', '那边', '也', '不加', '食品', '的']
True
[ 1.2495377   0.06170199  0.8938702   0.35700548  0.31723502 -0.61625624
  1.277664    1.202538   -1.1585867   0.33666825  1.109832   -1.652932
 -1.0610119  -1.0041544   0.806956    0.9353029  -0.05180022 -0.6503395
 -0.21520421  2.3639724   1.2721382   1.5534258   0.38471395 -1.9917885
  0.11107784  1.2311084  -0.13152413  0.03202014  0.17278504  0.06345499
 -0.59285796  0.9592659  -1.3649746  -1.0861205  -1.1999605   0.90548515
  0.9188584   0.90441847  0.6442876  -0.81567323 -0.985507    1.493433
  0.04943489 -1.2045292   0.5940871  -0.92582273 -0.19844496 -0.44449088
  0.02002396  0.30146676  0.33320138  0.4386125   0.513078    1.8808981
  2.0416908  -0.4570658  -0.15623297  0.14181511 -1.101433   -0.8858512
 -1.4654648  -1.2278835  -0.6489866   0.5554768 ]


In [40]:
# 计算词的TF-idf

# 统计type and token
wordList = {}
for sentence in sentences:
    for word in sentence:
        if word in wordList:
            wordList[word] += 1
        else:
            wordList[word] = 1;


In [41]:
import math
import numpy as np
def softmax(x):
    x = np.array(x)
    # 计算每行的最大值
    row_max = np.max(x)
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    x = x - row_max
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp)
    s = x_exp / x_sum
    return s

In [42]:

VectorSentence = []
for sentence in sentences:
    tmp = np.zeros(64)
    for word in sentence:
        # wordcurFreq = sentence.count(word)
        # wordAllFreq = wordList[word]
        weight = float(sentence.count(word))/len(sentence) * math.log(float(len(CBOW_Vector))/wordList[word])
        if word in CBOW_Vector:
            tmp += (CBOW_Vector[word] * weight)
        else:
            tmp += (np.zeros(64))
    
    VectorSentence.append(softmax(tmp))

In [43]:
np.save('句子的向量表示_bin',VectorSentence)

In [48]:
print(len(VectorSentence))
print(posLen)
print(negLen)

12361
5687
6674


In [44]:
# 存储
def GetLabel():
    dicPos = {}
    dicNeg = {}
    path = '.\\row_语料分词_'
    with open(path + 'pos.json','r',encoding = 'utf-8') as f:
        dicPos = json.load(f)
       
    with open(path + 'neg.json','r',encoding = 'utf-8') as f:
        dicNeg = json.load(f)
    return dicPos,dicNeg

dicPos,dicNeg = GetLabel()



In [45]:
print(len(dicPos))

2000


In [18]:
with open (r'./句子的向量表示.txt','w',encoding = 'utf-8') as f:
    cur = 0
    for i in range(0,len(dicPos)):
        f.write (str(i) + '\t' + str(list(VectorSentence[cur])) + '\n')
        cur += 1
    for iter in (VectorSentence[cur:]):
        f.write (str(i) + '\t' + str(list(iter)) + '\n')
        cur += 1